# Clustering with All-Or-Nothing Affinities

In this notebook we'll illustrate one of the most important practical cases for hypergraph modularity clustering, the case of all-or-nothing affinities. As we show in our paper

> Chodrow, Philip S., Nate Veldt, and Austin R. Benson. "Generative hypergraph clustering: from blockmodels to modularity." arXiv preprint arXiv:2101.09611 (2021),

this class of clustering problem admits highly performant Louvain-type algorithms. It also generalizes the "strict modularity" objective developed in

> Kamiński, Bogumił, et al. "Clustering via hypergraph modularity." PloS ONE 14.11 (2019).

The experiment we show here is a simplified version of the experiment on school contact networks shown in our paper in Fig. 3. 

In [1]:
using Revise
using Pkg; Pkg.activate("../.")
using HyperModularity

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /home/phil/.julia/packages/Revise/BqeJF/src/Revise.jl:1328
 Activating environment at `~/HyperModularity/Project.toml`


## Acquire Data

In [2]:
dataset = "contact-high-school-classes"
maxsize = 5	# max hyperedge size
minsize = 2	# min hyperedge size
return_labels = true
H, Z = read_hypergraph_data(dataset,maxsize,minsize,return_labels)

(hypergraph
  N: Array{Int64}((327,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  318, 319, 320, 321, 322, 323, 324, 325, 326, 327]
  E: Dict{Int64,Dict}
  D: Array{Int64}((327,)) [36, 44, 33, 72, 74, 66, 53, 63, 117, 35  …  45, 8, 38, 44, 12, 55, 16, 2, 64, 10]
, [9, 9, 3, 3, 8, 8, 3, 3, 7, 7  …  9, 4, 5, 9, 7, 9, 2, 1, 6, 4])

Although we have read in the true labels, we are going to treat them as unknown and use them only for final comparison. 

### Warmstart

We'd like to form an estimate of the AON affinity function $\Omega$, which requires a preliminary label estimate. For this we use dyadic Louvain: 

In [3]:
Ẑ = CliqueExpansionModularity(H);

Now we can estimate the AON affinity. To do so we use the function `estimateΩEmpirically`, with a custom `aggregator`. This `aggregator` is a function that maps a partition vector `p` to one or more features of `p`. In the case of the AON affinity, we assign parameters based on the following two features: 

1. Whether or not  `p` contains a single entry, which corresponds to a perfectly homogeneous hyperedge. 
2. The sum of `p`, which gives the size of the hyperedge. 

We can therefore perform our estimate like this: 

In [4]:
# AON aggregator
aggregator = p -> [length(p) == 1, sum(p)]

Ω̂ = estimateΩEmpirically(H, Ẑ; aggregator = aggregator);

Now we're ready to alternate between estimating labels and $\hat{\Omega}$. 

In [5]:
for i ∈ 1:10
    Ẑ = AON_Louvain(H, Ω̂; α = 0, verbose = false,  scan_order = "random")
    Ω̂  = estimateΩEmpirically(H, Ẑ; aggregator = aggregator)
    
    Q = round(Float64(modularity(H, Ẑ, Ω̂;α = 0)), digits = 2)
    k = length(unique(Ẑ))
    println("Current modularity $Q with $k clusters.")
end

Current modularity -112597.7 with 5 clusters.
Current modularity -112597.7 with 5 clusters.
Current modularity -112597.7 with 5 clusters.
Current modularity -112453.01 with 6 clusters.
Current modularity -112453.01 with 6 clusters.
Current modularity -112453.01 with 6 clusters.
Current modularity -112453.01 with 6 clusters.
Current modularity -110512.3 with 7 clusters.
Current modularity -109919.77 with 9 clusters.
Current modularity -109919.77 with 9 clusters.


In this relatively easy clustering problem, we perfectly recover the ground-truth clusters after 10 iterations. 

In [6]:
mutualInformation(Z, Ẑ, true)

1.0